In [105]:
from pathlib import Path
import numpy as np
import pandas as pd 
import json
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from plotly.offline import iplot

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook" 

from workbench.wandb import wandb_model_DB, make_clickable, color_state, get_architecture_from_model_DB_run_id, get_model_DB_run_id_from_architecture

import wandb



%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
%reload_ext autoreload
%autoreload

In [107]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [108]:
df  = wandb_model_DB()


In [109]:
df["peak_memory_kb"] = df["peak_memory_b"]/1024.0
df["flash_model_size_kb_INT8_mltk"] = df["flash_model_size_b__INT8_mltk"]/1024.0
df["RAM_runtime_memory_size_kb_INT8_mltk"] = df["RAM_runtime_memory_size_b_INT8_mltk"]/1024.0
df["alpha"] = df["alpha"].astype(str)
df.test_accuracy.fillna(0, inplace=True)
df['test_accuracy'] = df['test_accuracy'].replace("NaN", 0)
df.style.format({'url': make_clickable}).applymap(color_state, subset=["state"])

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk
0,fgxqsgfd,mobilenetv2_0.25_96_c3_o2_t5l512.MV1,finished,4.071 M,https://wandb.ai/susbrock/model_DB/runs/fgxqsgfd,mobilenetv2_0.25_96_c3_o2_t5l512.MV1,8.641 M,0.25,2,96,8909690,3,4071184,t5l512.MV1,0.005183,160382,9020290,1083.320312,57600.000000,4071184,0.005382,152654,7728,599.960938,614360,0,275.531250,282144,272032,0,100364,[],2023-02-26T13:15:12,"['susbrock', 'model_DB', 'fgxqsgfd']",None,0.000000,56.250000,275.531250,98.011719
1,vzkjx8vd,mobilenetv2_0.25_96_c3_o2_t5l256.MV1,finished,3.886 M,https://wandb.ai/susbrock/model_DB/runs/vzkjx8vd,mobilenetv2_0.25_96_c3_o2_t5l256.MV1,8.265 M,0.25,2,96,8530554,3,3886352,t5l256.MV1,0.005193,138366,8641154,997.320312,57600.000000,3886352,0.005392,131150,7216,516.960938,529368,0,248.031250,253984,269984,0,98316,[],2023-02-26T13:13:09,"['susbrock', 'model_DB', 'vzkjx8vd']",None,0.000000,56.250000,248.031250,96.011719
2,aj5kvuh0,mobilenetv2_0.25_96_c3_o2_t5l128.MV1,finished,3.794 M,https://wandb.ai/susbrock/model_DB/runs/aj5kvuh0,mobilenetv2_0.25_96_c3_o2_t5l128.MV1,8.076 M,0.25,2,96,8340986,3,3793936,t5l128.MV1,0.005131,127358,8451586,955.242188,57600.000000,3793936,0.005330,120398,6960,475.460938,486872,0,234.281250,239904,268960,0,97292,[],2023-02-26T13:11:08,"['susbrock', 'model_DB', 'aj5kvuh0']",None,0.832163,56.250000,234.281250,95.011719
3,exjgrqit,mobilenetv2_0.25_96_c3_o2_t6l128.MV1,finished,4.387 M,https://wandb.ai/susbrock/model_DB/runs/exjgrqit,mobilenetv2_0.25_96_c3_o2_t6l128.MV1,9.342 M,0.25,2,96,9646994,3,4386784,t6l128.MV1,0.005924,150250,9757594,1044.640625,69120.000000,4386784,0.006123,142106,8144,556.328125,569680,0,268.726562,275176,319764,0,113556,[],2023-02-26T13:09:03,"['susbrock', 'model_DB', 'exjgrqit']",None,0.000000,67.500000,268.726562,110.894531
4,dl8ye13s,mobilenetv2_0.25_96_c3_o2_t6l256.MV1,finished,4.479 M,https://wandb.ai/susbrock/model_DB/runs/dl8ye13s,mobilenetv2_0.25_96_c3_o2_t6l256.MV1,9.530 M,0.25,2,96,9836562,3,4479200,t6l256.MV1,0.005986,161258,9947162,1086.718750,69120.000000,4479200,0.006185,152858,8400,597.828125,612176,0,282.476562,289256,320788,0,114580,[],2023-02-26T13:06:58,"['susbrock', 'model_DB', 'dl8ye13s']",None,0.000000,67.500000,282.476562,111.894531
5,dburdcfy,mobilenetv2_0.25_96_c3_o2_t6l512.MV1,finished,4.664 M,https://wandb.ai/susbrock/model_DB/runs/dburdcfy,mobilenetv2_0.25_96_c3_o2_t6l512.MV1,9.907 M,0.25,2,96,10215698,3,4664032,t6l512.MV1,0.005975,183274,10326298,1172.718750,69120.000000,4664032,0.006175,174362,8912,680.828125,697168,0,309.976562,317416,322836,0,116628,[],2023-02-26T13:04:54,"['susbrock', 'model_DB', 'dburdcfy']",None,0.836906,67.500000,309.976562,113.894531
6,z7yhusdp,mobilenetv2_0.2_96_c3_o2_t6l512.MV1,finished,3.192 M,https://wandb.ai/susbrock/model_DB/runs/z7yhusdp,mobilenetv2_0.2_96_c3_o2_t6l512.MV1,6.833 M,0.2,2,96,7070072,3,3192010,t6l512.MV1,0.004611,127826,7180672,955.523438,55296.000000,3192010,0.004811,120662,7164,477.406250,488864,0,238.085938,243800,246908,0,95600,[],2023-02-26T13:01:50,"['susbrock', 'model_DB', 'z7yhusdp']",None,0.000000,54.000000,238.085938,93.359375
7,azckjhgk,mobilenetv2_0.2_96_c3_o2_t6l256.MV1,finished,3.044 M,https://wandb.ai/susbrock/model_DB/runs/azckjhgk,mobilenetv2_0.2_96_c3_o2_t6l256.MV1,6.530 M,0.2,2,96,6764664,3,3044042,t6l256.MV1,0.004566,109906,6875264,885.523438,55296.000000,3044042,0.004765,103254,6652,410.406250,420256,0,214.585938,219736,244860,0,93552,[],2023-02-26T12:59:17,"['su

In [36]:
df.test_accuracy.value_counts

<bound method IndexOpsMixin.value_counts of 0     0.000000
1     0.000000
2     0.832163
3     0.000000
4     0.000000
5     0.836906
6     0.000000
7     0.815744
8     0.814301
9     0.805984
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.748791
19    0.763021
20    0.000000
21    0.000000
22    0.000000
23    0.000000
24    0.000000
25    0.000000
26    0.000000
27    0.833440
28    0.833531
29    0.000000
30    0.000000
31    0.836906
32    0.872936
33    0.000000
34    0.000000
35    0.799872
36    0.000000
37    0.771231
38    0.833531
39    0.851865
40    0.851136
41    0.000000
42    0.828697
43    0.000000
44    0.812825
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49    0.000000
50    0.000000
51    0.850771
52    0.838913
53    0.801879
54    0.000000
55    0.000000
56    0.000000
57    0.000000
58    0.000000
59    0.000000
60    0.000000
61    0.000000
62    0.000000
63    0.000

In [110]:
df.columns

Index(['id', 'run_name', 'state', 'MACs', 'url', 'architecture', 'FLOPs',
       'alpha', 'classes', 'img_res', 'OPS_mltk', 'channels', 'macs_mltk',
       'variation', 'energy_mltk', 'total_params', 'OPS_INT8_mltk',
       'model_size_kb', 'peak_memory_b', 'macs_INT8_mltk', 'energy_INT8_mltk',
       'trainable_params', 'non_trainable_params', 'tflite_model_size_kb',
       'flash_model_size_b_mltk', 'n_unsupported_layers_mltk',
       'tflite_model_INT8_size_kb', 'flash_model_size_b__INT8_mltk',
       'RAM_runtime_memory_size_b_mltk', 'n_unsupported_layers_INT8_mltk',
       'RAM_runtime_memory_size_b_INT8_mltk', 'tags', 'created_at', 'path',
       'notes', 'test_accuracy', 'peak_memory_kb',
       'flash_model_size_kb_INT8_mltk',
       'RAM_runtime_memory_size_kb_INT8_mltk'],
      dtype='object')

In [111]:
# check for redundant models in the model_DB
df["architecture"].value_counts()

mobilenetv2_0.25_96_c3_o2_t5l512.MV1       1
mobilenetv2_0.25_96_c3_o2_t5l256.MV1       1
mobilenetv2_0.35_96_c3_o2_l5               1
mobilenetv2_0.5_96_c3_o2_l5                1
mobilenetv2_0.2_96_c3_o2_l5                1
mobilenetv2_0.1_96_c3_o2_l5                1
mobilenetv1vvw_0.25_96_c3_o2_vvw           1
mobilenetv1_0.1_96_c3_o2_l5.GAP            1
shufflenetv1_0.25_96_c3_o2_g8              1
shufflenetv1_0.25_96_c3_o2_g1              1
mnasnet_1_96_c3_o2_keras                   1
shufflenetv2_1_96_c3_o2_nosplit            1
shufflenetv2_1_96_c3_o2_000                1
shufflenetv2_0.5_96_c3_o2_000              1
shufflenetv2tiny_0.5_96_c3_o2_f24l1024     1
shufflenetv2tiny_0.35_96_c3_o2_f24l1024    1
shufflenetv2tiny_0.25_96_c3_o2_f24l1024    1
shufflenetv2tiny_0.2_96_c3_o2_f24l1024     1
shufflenetv2tiny_0.1_96_c3_o2_f24l1024     1
mobilenetv2_0.25_96_c3_o2_l5               1
mobilenetv2_1_96_c3_o2_l5                  1
mobilenetv2_1_96_c3_o2_keras               1
mobilenetv

# Helper functions

In [112]:
# def get_model_DB_run_id_from_architecture(architecture):
#     df  = wandb_model_DB()

#     run_id = df.query(f"architecture=='{architecture}'")["id"].values[0]
#     return run_id

In [113]:
id = get_model_DB_run_id_from_architecture("efficientNetB0_0.05_96_c3_o2_keras")
id

'ndi1k023'

In [114]:
# get_architecture_from_model_DB_run_iddef (run_id):
#     # get model name from run_id
#     df  = wandb_model_DB()
#     model_name = df.query(f"id=='{run_id}'")['architecture'].values[0]
#     return model_name

In [115]:
myname = get_architecture_from_model_DB_run_id(id)
myname

'efficientNetB0_0.05_96_c3_o2_keras'

In [116]:
training_csv_path = Path.cwd().joinpath("model_DB_visual_wake_words.csv")
training_csv_path.is_file()
# df = pd.read_csv(training_csv_path)
# df

True

# Plotting functions

In [45]:
def get_model_family_df(model_family):

    
    model_list = df[["architecture"]].query(f"architecture.str.contains('{model_family}', na=False)").copy()
    model_list = model_list["architecture"].tolist()
    #print(model_list)
    family_df = df.query("architecture in @model_list").copy()
    family_df.sort_values(by="alpha", ascending=True, inplace=True)
    print(f"Number of models: {len(family_df)}")
    
    return family_df

In [56]:
def memory_fig(df_long):

    peak_memory_limit = 87
    flash_memory_limit = 250
    fig = px.bar(df_long, x="architecture", color="size",
                y='value',
                labels={"architecture": "model architecture",
                            #"size": "size",
                            #"value": "kB"
                            },
                title="Peak memory, RAM size & flash size",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=600
                )
    fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
                annotation_position="top left")
    fig.add_hline(y=flash_memory_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{flash_memory_limit} KB flash memory", 
                annotation_position="top right")
    fig.show()
    return fig

In [59]:
def create_size_df_long(df):
    size_df = df[["architecture", 
             "peak_memory_kb", 
             "RAM_runtime_memory_size_kb_INT8_mltk",
             "flash_model_size_kb_INT8_mltk",
             "tflite_model_INT8_size_kb"
             ]].copy()
    df_long = pd.melt(size_df, id_vars=["architecture"], var_name= "size", value_name="value").sort_values(by=["architecture","size"])
    return df_long

# MobileNetv V1 Analysis

In [44]:
# mobilenetv1_list = df[["architecture"]].query("architecture.str.contains('mobilenetv1', na=False)").copy()
# mobilenetv1_list = mobilenetv1_list["architecture"].tolist()
# mobilenetv1_list

In [104]:
mbnv1_df = get_model_family_df("mobilenetv1")
mbnv1_df.head()

Number of models: 21


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk
9,f6keiwns,mobilenetv1_0.1_96_c3_o2_l2.MV1,finished,1.117 M,https://wandb.ai/susbrock/model_DB/runs/f6keiwns,mobilenetv1_0.1_96_c3_o2_l2.MV1,2.467 M,0.1,2,96,2467123,3,1116555,l2.MV1,0.001475,30461,2577723,321.664062,55296.0,1116555,0.001675,28905,1556,119.562500,122432,0,64.015625,65552,149096,0,66700,[],2023-02-26T12:52:37,"[susbrock, model_DB, f6keiwns]",None,0.805984,54.0,64.015625,65.136719
33,lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,finished,7.877 M,https://wandb.ai/susbrock/model_DB/runs/lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,17.203 M,0.1,2,224,17203193,3,7877395,l5.GAP,0.017652,41171,17805313,418.878906,301056.0,7877395,0.017902,39003,2168,159.527344,163356,0,85.500000,87552,767088,0,316080,[],2023-02-22T20:34:10,"[susbrock, model_DB, lum2673e]",None,0.000000,294.0,85.500000,308.671875
34,9s1mqwqn,mobilenetv1_0.1_224_c3_o2_l5.MV1,finished,7.877 M,https://wandb.ai/susbrock/model_DB/runs/9s1mqwqn,mobilenetv1_0.1_224_c3_o2_l5.MV1,17.208 M,0.1,2,224,17208291,3,7877395,l5.MV1,0.017608,41171,17810411,416.707031,301056.0,7877395,0.017858,39003,2168,159.207031,163028,0,85.132812,87176,767000,0,315996,[],2023-02-22T20:28:52,"[susbrock, model_DB, 9s1mqwqn]",None,0.000000,294.0,85.132812,308.589844
35,t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,3.161 M,0.1,2,96,3161131,3,1447035,l5.MV1,0.001656,41171,3271731,416.699219,55296.0,1447035,0.001855,39003,2168,159.207031,163028,0,85.132812,87176,152600,0,70236,[],2023-02-22T19:53:41,"[susbrock, model_DB, t3imun41]",None,0.799872,54.0,85.132812,68.589844
53,sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,3.160 M,0.1,2,96,3160113,3,1447035,l5.GAP,0.001695,41171,3270713,418.722656,55296.0,1447035,0.001895,39003,2168,159.320312,163144,0,85.281250,87328,152688,0,70320,[],2023-02-18T10:08:46,"[susbrock, model_DB, sd3qtst4]",None,0.801879,54.0,85.281250,68.671875


In [65]:
mbnv1_size_df_long = create_size_df_long(mbnv1_df)
mbnv1_memory_fig = memory_fig(mbnv1_size_df_long)

# MobileNet V2 Analysis

In [103]:
mbnv2_df = get_model_family_df("mobilenetv2")
mbnv2_df.head()

Number of models: 40


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk
20,f8jsvkj6,mobilenetv2_0.1_96_c3_o2_t3l1024.MV1,finished,1.298 M,https://wandb.ai/susbrock/model_DB/runs/f8jsvkj6,mobilenetv2_0.1_96_c3_o2_t3l1024.MV1,2.782 M,0.1,2,96,2876085,3,1298435,t3l1024.MV1,0.002935,53810,2986685,666.367188,55296.0,1298435,0.003134,50084,3726,215.406250,220576,0,132.890625,136080,210464,0,82356,[],2023-02-25T10:40:40,"[susbrock, model_DB, f8jsvkj6]",None,0.0,54.0,132.890625,80.425781
23,l2ngjsr1,mobilenetv2_0.1_96_c3_o2_t6l1024.MV1,finished,1.618 M,https://wandb.ai/susbrock/model_DB/runs/l2ngjsr1,mobilenetv2_0.1_96_c3_o2_t6l1024.MV1,3.500 M,0.1,2,96,3633732,3,1617926,t6l1024.MV1,0.003311,67766,3744332,720.851562,55296.0,1617926,0.003511,62684,5082,259.789062,266024,0,159.921875,163760,215688,0,87500,[],2023-02-25T10:34:05,"[susbrock, model_DB, l2ngjsr1]",None,0.0,54.0,159.921875,85.449219
22,2vtocbic,mobilenetv2_0.1_96_c3_o2_t5l1024.MV1,finished,1.511 M,https://wandb.ai/susbrock/model_DB/runs/2vtocbic,mobilenetv2_0.1_96_c3_o2_t5l1024.MV1,3.261 M,0.1,2,96,3381183,3,1511429,t5l1024.MV1,0.003114,63114,3491783,703.601562,55296.0,1511429,0.003314,58484,4630,245.085938,250968,0,150.882812,154504,214140,0,85940,[],2023-02-25T10:36:19,"[susbrock, model_DB, 2vtocbic]",None,0.0,54.0,150.882812,83.925781
55,r7tokmwg,mobilenetv2_0.1_96_c3_o2_l5,finished,1.692 M,https://wandb.ai/susbrock/model_DB/runs/r7tokmwg,mobilenetv2_0.1_96_c3_o2_l5,3.643 M,0.1,2,96,3778886,3,1692166,l5,0.003322,77494,3889486,760.898438,55296.0,1692166,0.003521,71900,5594,295.679688,302776,0,176.492188,180728,218192,0,108936,[],2023-02-11T21:12:26,"[susbrock, model_DB, r7tokmwg]",None,0.0,54.0,176.492188,106.382812
21,6oq96cks,mobilenetv2_0.1_96_c3_o2_t4l1024.MV1,finished,1.405 M,https://wandb.ai/susbrock/model_DB/runs/6oq96cks,mobilenetv2_0.1_96_c3_o2_t4l1024.MV1,3.021 M,0.1,2,96,3128634,3,1404932,t4l1024.MV1,0.003029,58462,3239234,684.351562,55296.0,1404932,0.003228,54284,4178,230.234375,235760,0,141.710938,145112,211848,0,83688,[],2023-02-25T10:38:38,"[susbrock, model_DB, 6oq96cks]",None,0.0,54.0,141.710938,81.726562


In [68]:

macs_params_fig = px.scatter(
    mbnv2_df,
    x="macs_mltk",
    y="tflite_model_INT8_size_kb",
    color="alpha",
    size="total_params",
    hover_data=["run_name","alpha", "macs_mltk", "total_params"],
    labels={"macs_mltk": "MACs", 
            "tflite_model_INT8_size_kb" : "INT8 model size in kB",
            "total_params": "parameters"},
    title=f"INT8 model size vs. MACs",
)

macs_params_fig.update_layout(showlegend=True, width=500, height=400)

macs_params_fig.show()

In [69]:
def params_macs_alpha_fig(df, model_name):
    df["alpha"] = df["alpha"].astype(str)
    fig = px.scatter(
        df,
        x="total_params",
        y="macs_mltk",
        color="alpha",
        hover_data=["run_name","alpha", "macs_mltk", "total_params"],
        size="tflite_model_INT8_size_kb",
        labels={"total_params": "parameters", 
                "macs_mltk": "MACs",
                "run_name": "model name"},
        title=f"MACs vs. total parameters - {model_name}",
    )
    fig.add_hline(y=60000000, line_width=2, line_dash="dash", line_color="red", annotation_text=f"60M MACs", 
               annotation_position="top")
    fig.update_layout(showlegend=True, width=1200, height=400)
    
    fig.show()
    return fig


In [70]:
mbnv2_fig = params_macs_alpha_fig(mbnv2_df, "MobileNet V2")

In [94]:
flops_params_fig = px.scatter(
    mbnv2_df,
    x="total_params",
    y="OPS_INT8_mltk",
    color="alpha",
    size="tflite_model_INT8_size_kb",
    hover_data=["run_name","alpha", "macs_mltk", "total_params"],
    labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"FLOPs vs. total parameters",
)
flops_params_fig.update_layout(showlegend=True, width=500, height=400)

flops_params_fig.show()

In [73]:
# df["peak_memory_kb"] = df["peak_memory_b"]/1024.0
# df["flash_model_size_kb_INT8_mltk"] = df["flash_model_size_b__INT8_mltk"]/1024.0
# df["RAM_runtime_memory_size_kb_INT8_mltk"] = df["RAM_runtime_memory_size_b_INT8_mltk"]/1024.0

In [74]:
mbnv2_size_df_long = create_size_df_long(mbnv2_df)
mobilenetv2_mem_fig = memory_fig(mbnv2_size_df_long)

# Plotting construction zone

In [ ]:
# size_fig = px.bar(mbnv2_df, x="alpha", y= ["peak_memory_kb",
#                                            "RAM_runtime_memory_size_kb_INT8_mltk",
#                                            "flash_model_size_kb_INT8_mltk",
#                                            "tflite_model_INT8_size_kb"], 
#                   text_auto=".2s",
#                   labels={"name" : "layer",
#                           "macs" : "MACs"},
#                      width=1200, height=600,
#     title=f"Model size")
# size_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#             annotation_position="bottom right")
# #peak_memory = peak_memory_df["RAM_b"].max()
# #peak_mem_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} B peak activation", 
# #               annotation_position="top")
# # tflite_fig.update_layout(yaxis_range=[0,300])
# size_fig.update_traces(width=1)
# size_fig.update_layout(showlegend=True)

# size_fig.show()

In [ ]:
# # Create figure with secondary y-axis
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# #fig = go.Figure()#specs=[[{"secondary_y": True}]])
# # Add traces
# fig.add_trace(go.Bar(x = mbnv2_df["alpha"],
#                                    y = mbnv2_df["tflite_model_INT8_size_kb"],#, mbnv2_df["peak_memory_b"]], 
#                                    name= "model size in kB"
#                                    ),
#                                    col=1,
#                                    row=1,
#                                    secondary_y=False)

# fig.add_trace(go.Bar(x = mbnv2_df["alpha"],
#                                    y = mbnv2_df["peak_memory_b"], 
#                                    name= "peak memory in B"
#                                    ),
#                                    col=1,
#                                    row=1,
#                                    secondary_y=True)



# # fig.add_trace(
# #     go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
# #     secondary_y=True,
# # )

# # Add figure title
# fig.update_layout(
#     title_text="Double Y Axis Example"
# )

# # Set x-axis title
# fig.update_xaxes(title_text="xaxis title")

# # Set y-axes titles
# fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
# fig.update_layout(barmode="group",
#                   bargap=0.15, # gap between bars of adjacent location coordinates.
#                     bargroupgap=0.1 # gap between bars of the same location coordinate.)
# )
# fig.show()

wandb.log({"my_chart": wandb.Html(plotly.io.to_html(fig)})

# ShuffleNet V2 tiny Analysis

In [85]:
shufflenetv2tiny_list = df[["architecture"]].query("architecture.str.contains('shufflenetv2tiny')").copy()
shufflenetv2tiny_list = shufflenetv2tiny_list["architecture"].tolist()
shufflenetv2tiny_list

['shufflenetv2tiny_0.05_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.25_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.25_96_c3_o2_f24l512',
 'shufflenetv2tiny_0.05_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.2_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.25_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.35_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.5_96_c3_o2_f24l1024']

In [87]:
shufflev2tiny_df = get_model_family_df("shufflenetv2tiny")
shufflev2tiny_df

Number of models: 10


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk
37,zmlnt7qa,shufflenetv2tiny_0.05_96_c3_o2_f24l128,finished,1.900 M,https://wandb.ai/susbrock/model_DB/runs/zmlnt7qa,shufflenetv2tiny_0.05_96_c3_o2_f24l128,4.073 M,0.05,2,96,4160826,3,1899760,f24l128,0.003729,20802,4271426,786.578125,80664.0,1899760,0.003928,19254,1548,135.328125,138576,0,119.015625,121872,357736,0,117296,[],2023-02-20T19:14:20,"[susbrock, model_DB, zmlnt7qa]",None,0.771231,78.773438,119.015625,114.546875
41,x4xe6px9,shufflenetv2tiny_0.05_96_c3_o2_f24l1024,finished,2.289 M,https://wandb.ai/susbrock/model_DB/runs/x4xe6px9,shufflenetv2tiny_0.05_96_c3_o2_f24l1024,4.851 M,0.05,2,96,4946618,3,2288624,f24l1024,0.003780,65602,5057218,962.578125,80664.0,2288624,0.003979,64054,1548,313.828125,321360,0,187.265625,191760,364904,0,124464,[],2023-02-19T14:15:09,"[susbrock, model_DB, x4xe6px9]",None,0.000000,78.773438,187.265625,121.546875
38,287svej7,shufflenetv2tiny_0.1_96_c3_o2_f24l128,finished,2.741 M,https://wandb.ai/susbrock/model_DB/runs/287svej7,shufflenetv2tiny_0.1_96_c3_o2_f24l128,5.799 M,0.1,2,96,5916690,3,2741080,f24l128,0.003817,56058,6027290,927.585938,80664.0,2741080,0.004017,53058,3000,261.941406,268228,0,167.859375,171888,363356,0,122944,[],2023-02-20T19:05:05,"[susbrock, model_DB, 287svej7]",None,0.833531,78.773438,167.859375,120.062500
42,d1jdmubc,shufflenetv2tiny_0.1_96_c3_o2_f24l1024,finished,3.517 M,https://wandb.ai/susbrock/model_DB/runs/d1jdmubc,shufflenetv2tiny_0.1_96_c3_o2_f24l1024,7.351 M,0.1,2,96,7476626,3,3517016,f24l1024,0.003940,143866,7587226,1271.585938,80664.0,3517016,0.004139,140866,3000,608.441406,623044,0,278.109375,284784,370524,0,130112,[],2023-02-19T14:12:31,"[susbrock, model_DB, d1jdmubc]",None,0.828697,78.773438,278.109375,127.062500
43,e0bbs9ge,shufflenetv2tiny_0.2_96_c3_o2_f24l1024,finished,7.187 M,https://wandb.ai/susbrock/model_DB/runs/e0bbs9ge,shufflenetv2tiny_0.2_96_c3_o2_f24l1024,14.777 M,0.2,2,96,14962754,3,7186856,f24l1024,0.005244,347698,15073354,2066.468750,80664.0,7186856,0.005443,341794,5904,1382.375000,1415552,0,505.921875,518064,382160,0,141748,[],2023-02-19T14:09:48,"[susbrock, model_DB, e0bbs9ge]",None,0.000000,78.773438,505.921875,138.425781
39,l9yu765d,shufflenetv2tiny_0.25_96_c3_o2_f24l128,finished,6.961 M,https://wandb.ai/susbrock/model_DB/runs/l9yu765d,shufflenetv2tiny_0.25_96_c3_o2_f24l128,14.355 M,0.25,2,96,14552658,3,6961432,f24l128,0.004929,227162,14663258,1593.992188,80664.0,6961432,0.005128,220290,6872,900.687500,922304,0,373.265625,382224,378820,0,138420,[],2023-02-20T19:01:34,"[susbrock, model_DB, l9yu765d]",None,0.851865,78.773438,373.265625,135.175781
40,r7tncy00,shufflenetv2tiny_0.25_96_c3_o2_f24l512,finished,7.736 M,https://wandb.ai/susbrock/model_DB/runs/r7tncy00,shufflenetv2tiny_0.25_96_c3_o2_f24l512,15.905 M,0.25,2,96,16105938,3,7736344,f24l512,0.005283,313946,16216538,1933.992188,80664.0,7736344,0.005482,307074,6872,1241.187500,1270976,0,468.515625,479760,381892,0,141492,[],2023-02-19T14:17:41,"[susbrock, model_DB, r7tncy00]",None,0.851136,78.773438,468.515625,138.175781
44,1hl3dr3g,shufflenetv2tiny_0.25_96_c3_o2_f24l1024,finished,8.770 M,https://wandb.ai/susbrock/model_DB/runs/1hl3dr3g,shufflenetv2tiny_0.25_96_c3_o2_f24l1024,17.971 M,0.25,2,96,18176978,3,8769560,f24l1024,0.005623,429658,18287578,2385.992188,80664.0,8769560,0.005822,422786,6872,1695.187500,1735872,0,595.515625,609808,385988,0,145588,[],2023-02-19T14:07:23,"[susbrock, model_DB, 1hl3dr3g]",None,0.812

In [88]:
shufflev2tiny_df_long = create_size_df_long(shufflev2tiny_df)
shufflev2tiny_mem_fig = memory_fig(shufflev2tiny_df_long)

In [101]:
### TODO: Clean up accuracy plots
macs_accuracy_fig = px.scatter(
    shufflev2tiny_df,
    x="MACs",
    y="test_accuracy",
    color="alpha",
    hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
    #size="tflite_model_INT8_size_kb",
    #labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"Accuracy vs. MACs - ShuffleNetV2Tiny",
)
macs_accuracy_fig.update_layout(showlegend=True, width=1200, height=400)

macs_accuracy_fig.show()

In [102]:
### TODO: Clean up accuracy plots
macs_accuracy_fig = px.scatter(
    mbnv2_df,
    x="MACs",
    y="test_accuracy",
    color="alpha",
    hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
    #size="tflite_model_INT8_size_kb",
    #labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"Accuracy vs. MACs",
)
macs_accuracy_fig.update_layout(showlegend=True, width=1200, height=400)

macs_accuracy_fig.show()

# Analysis of single architectures

In [75]:
architecture = "mobilenetv1_0.1_96_c3_o2_l2.MV1"

In [76]:
run_id =  get_model_DB_run_id_from_architecture(architecture)
run_id

'f6keiwns'

In [77]:
# get model name from run_id
model_name = df.query(f"id=='{run_id}'")['architecture'].values[0]
model_name

'mobilenetv1_0.1_96_c3_o2_l2.MV1'

# Comparison of two models

In [78]:
architecture_1 = "mobilenetv2_0.25_96_c3_o2_l5"
run_id_1 = get_model_DB_run_id_from_architecture(architecture_1)
architecture_2 = "mobilenetv2_0.25_96_c3_o2_keras"
run_id_2 = get_model_DB_run_id_from_architecture(architecture_2)

In [79]:
run_id_2

'upj2bfr1'

In [80]:
ENTITY = "susbrock"
PROJECT ="model_DB"
#run_id = "27bt3quc"
table_name = "tflite_layers_mltk"

In [81]:
df_me = get_wandb_table_as_df(run_id_1, table_name)
df_me.head()

NameError: name 'get_wandb_table_as_df' is not defined

In [ ]:
df_keras = get_wandb_table_as_df(run_id_2, table_name)
df_keras.head()

In [ ]:
macs_fig = px.bar(df_me, x="name", y= "macs",
                  text_auto=".2s",
                  labels={"name" : "layer",
                          "macs" : "MACs"},
                     width=1200, height=600,
    title=f"MACs per layer - tflite {architecture_1}")
# tflite_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
#peak_memory = peak_memory_df["RAM_b"].max()
#peak_mem_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} B peak activation", 
#               annotation_position="top")
# tflite_fig.update_layout(yaxis_range=[0,300])
macs_fig.update_traces(width=1)
macs_fig.update_layout(showlegend=False)

macs_fig.show()

In [ ]:


# dict for the dataframes and their names
dfs = {architecture_1 : df_me, architecture_2 : df_keras}

# plot the data
#fig = go.Figure(layout=dict(title=dict(text="MACs per layer")))
fig = go.Figure(layout_title_text="MACs per layer")
#fig = make_subplots(rows=2, cols=1, start_cell="bottom-left")

for i in dfs:
    fig = fig.add_trace(go.Bar(x = dfs[i]["index"],
                                   y = dfs[i]["macs"], 
                                   name = i))
fig.show()

In [ ]:
shufflenetv1_df = df.query("architecture in @shufflenetv1_list").copy()
shufflenetv1_df.sort_values(by="alpha", ascending=True, inplace=True)
shufflenetv1_df

In [ ]:
#run_name = "shufflenetv1_0.2_96_c3_o2_g3"

In [ ]:
# for shufflenet v1 
def extract_groups_from_name(architecture_name):

    groups = int(architecture_name.split("_")[-1].replace("g", ""))
    return groups

In [ ]:
shufflenetv1_df["groups"] = shufflenetv1_df["architecture"].apply(extract_groups_from_name)
shufflenetv1_df

In [ ]:
def params_macs_alpha_groups_fig(df,model_name):
    df["alpha"] = df["alpha"].astype(str)
    df["groups"] = df["groups"].astype(str)
    fig = px.scatter(
        df,
        x="total_params",
        y="macs_mltk",
        color="groups",
        hover_data=["alpha","groups", "macs_mltk", "total_params"],
        symbol= df["alpha"],
        #size="tflite_model_INT8_size_kb",
        labels={"total_params": "parameters", "macs_mltk": "MACs"},
        title=f"MACs vs. total parameters - {model_name}",
    )
    fig.add_hline(y=60000000, line_width=3, line_dash="dash", line_color="red", annotation_text=f"60M MACs", 
               annotation_position="top")
    fig.update_layout(showlegend=True, width=500, height=400)
    
    fig.show()
    return fig

In [ ]:
shufflev1_fig2 = params_macs_alpha_groups_fig(shufflenetv1_df, "ShuffleNet V1")

In [ ]:
shufflev1_size_df = shufflenetv1_df[["architecture", 
             "peak_memory_kb", 
             "RAM_runtime_memory_size_kb_INT8_mltk",
             "flash_model_size_kb_INT8_mltk",
             "tflite_model_INT8_size_kb"
             ]].copy()

In [82]:
# fig = make_subplots(rows=2, cols=1, start_cell="bottom-left", column_widths=[1400],row_heights=[0.5, 0.5], vertical_spacing=0.3,
#                     subplot_titles=[architecture_1, architecture_2])
# fig = fig.add_trace(go.Bar(x = df_me["name"],
#                                    y = df_me["macs"], 
#                                    name = architecture_1,
                                   
#                                    text=df_me["macs"],
#                                    textposition='auto'),
#                                    row=1, col=1)
# fig = fig.add_trace(go.Bar(x = df_keras["name"],
#                                    y = df_keras["macs"], 
#                                    name = architecture_2),
#                                    row=2, col=1)

# fig.update_layout(
#     height = 1000,
#     width = 1200,
#     title="MACs per layer - tflite",
#     xaxis_title="layer name",
#     yaxis_title="MACs",
#     legend_title="architecture",
#     # font=dict(
#     #     family="Courier New, monospace",
#     #     size=18,
#     #     color="RebeccaPurple"
#     # )
#     legend=dict(
#     x=0,
#     y=1.0
#     )
# )


# fig.update_layout(showlegend=False)
# fig.show()

NameError: name 'df_me' is not defined

In [ ]:
ENTITY = "susbrock"
PROJECT = "model_DB"
run_id_3 = "fbfsftzr"
table_name ="Runorder"

api = wandb.Api()
artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
artifact_dir = artifact.download()

In [ ]:
def get_wandb_node_type_summary_df(run_id):
    table_name = "Nodetypesummary"
    api = wandb.Api()
    artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
    artifact_dir = artifact.download()
    table_path = Path(artifact_dir).joinpath("Node type summary.table.json")
    #print(f"filepath {table_path}")

    with open(table_path) as file:
        json_dict = json.load(file)
        #print(json_dict)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])

    return df

In [ ]:
node_type_summary_df = get_wandb_node_type_summary_df(run_id)
node_type_summary_df

In [ ]:
peak_memory_df = get_wandb_table_as_df(run_id_3, "peak_memory")
peak_memory_df.head()

In [ ]:
RAM_df = peak_memory_df[["index", "name", "RAM_b"]]
RAM_df.head()

In [ ]:
tensor_df = get_wandb_table_as_df(run_id_3, "tensor_info")
tensor_df.drop(0, axis=0, inplace=True)
tensor_df.reset_index(inplace=True)
tensor_df.head()

In [ ]:
size_df = tensor_df[["index", "name", "size_b"]]
size_df.head()

In [ ]:
INT8_layers_df = get_wandb_table_as_df(run_id_3, "tflite_INt8_layers_mltk")
INT8_layers_df.head()

In [ ]:
macs_df = INT8_layers_df[["index", "name", "ops", "macs"]]
macs_df.head()

In [ ]:
def get_wandb_run_order_df(run_id):
    table_name = "Runorder"
    api = wandb.Api()
    artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
    artifact_dir = artifact.download()
    table_path = Path(artifact_dir).joinpath("Run order.table.json")
    #print(f"filepath {table_path}")

    with open(table_path) as file:
        json_dict = json.load(file)
        #print(json_dict)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])

    return df

In [ ]:
run_order_df = get_wandb_run_order_df(run_id_3)
run_order_df.reset_index(inplace=True)
run_order_df.head()

In [ ]:
time_df = run_order_df[["index", "node type", "first", "avg ms", "%"]]
time_df.head()

In [ ]:
combined_df = time_df.join(macs_df, on="index", rsuffix="_time")
combined_df = combined_df.join(RAM_df, on="index", rsuffix="_RAM" )
combined_df = combined_df.join(size_df, on="index", rsuffix="_size")
combined_df.head()

In [ ]:
combined_df["time_%"] = (combined_df['avg ms'] / combined_df['avg ms'].sum()) * 100
combined_df["RAM_%"] = (combined_df['RAM_b'] / combined_df['RAM_b'].sum()) * 100
combined_df["size_%"] = (combined_df['size_b'] / combined_df['size_b'].sum()) * 100
combined_df["ops_%"] = (combined_df['ops'] / combined_df['ops'].sum()) * 100
combined_df

In [ ]:
### TODO: add parameters per layer
### TODO: create a function for this fig

trace1 = go.Bar(
    x=combined_df["name"],
    y=combined_df["time_%"],
    name="time %",
    marker=dict(
        color='grey'
               )
)
trace2 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['RAM_%'],
    name='RAM %',
    #yaxis='y2',
    marker_symbol='star',
    marker_color="black",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

trace3 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['ops_%'],
    name='ops %',
    #yaxis='y2',
    marker_symbol='circle',
    marker_color="blue",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

trace4 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['size_%'],
    name='size %',
    #yaxis='y2',
    marker_symbol='cross', 
    marker_color="orange",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

fig= make_subplots()
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
#fig.add_trace(trace2,secondary_y=True)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.add_trace(trace4)
fig['layout'].update(height = 600, width = 1200, title ="Percentage per layer based on complete model - shufflenetv2tiny_0.5_96_c3_o2_f24l1024",xaxis=dict(
      tickangle=-90
    ))
fig.show()

In [ ]:
fig = px.bar(combined_df, x=combined_df['index'], y=combined_df['avg ms'])
fig.add_scatter(x=combined_df["index"], y=combined_df["RAM_b"])
fig.show()

In [ ]:
trace1 = go.Bar(
    x=combined_df["index"],
    y=combined_df["avg ms"],
    name="avg ms",
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=combined_df["index"],
    y=combined_df['RAM_b'],
    name='RAM',
    yaxis='y2',
    marker_symbol='circle',
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 1200, title ="Test",xaxis=dict(
      tickangle=-90
    ))
fig.show()

In [ ]:

# dict for the dataframes and their names
dfs = {architecture_1 : run_order_df}


fig = go.Figure(layout_title_text="average runtime in micro seconds per layer")
#fig = make_subplots(rows=2, cols=1, start_cell="bottom-left")

for i in dfs:
    fig = fig.add_trace(go.Bar(x = dfs[i]["index"],
                                   y = dfs[i]["avg ms"], 
                                   name = i,
                                   hoverinfo="all"
                                   #hoverinfo=dfs[i]["Name"]
                                   )
                                   )
            
fig.show()